In [2]:
from dateutil.parser import parse
from datetime import datetime
import numpy as np
import pandas as pd
import time

In [50]:
data_006 = pd.read_csv('/content/drive/capstone dataset/cleaned_2018_06.csv')
data_007 = pd.read_csv('/content/drive/capstone dataset/cleaned_2018_07.csv')
data_008 = pd.read_csv('/content/drive/capstone dataset/cleaned_2018_08.csv')

In [ ]:
data1 = data_006[data_006['tpep_pickup_datetime']>'2018-06-30 23:46:00']
data2 = data_007[data_007['tpep_pickup_datetime']<'2018-07-01 00:14:00']
data3 = data_007[data_007['tpep_pickup_datetime']>'2018-07-31 23:46:00']
data4 = data_008[data_008['tpep_pickup_datetime']<'2018-08-01 00:14:00']
data_add_06 = pd.concat([data1,data2])
data_add_07 = pd.concat([data3,data4])

In [5]:
def minutes_trans(newData):
  newData['tpep_pickup_datetime'] = newData['tpep_pickup_datetime'].map(lambda x: parse(x).strftime('%Y/%m/%d %H:%M'))
  newData['tpep_pickup_datetime'] = newData['tpep_pickup_datetime'].map(lambda x: datetime.strptime(x,'%Y/%m/%d %H:%M'))
  return newData

In [51]:
#transfer data of each month
newData = minutes_trans(data_008)

In [ ]:
#transfer appended last 15minutes 
newData_06_append = minutes_trans(data_add_06)
newData_06_append.reset_index(drop=True,inplace=True)
newData_07_append = minutes_trans(data_add_07)
newData_07_append.reset_index(drop=True,inplace=True)

In [7]:
# generate time series
def generate_countinous(start ='6/01/2018', end ='7/1/2018'):
  test_date = pd.date_range(start,end, freq='T')
  test_date = pd.DataFrame(test_date)
  test_date.rename(columns = {0:'pickup_time_g'}, inplace = True)
  test_date.drop([len(test_date)-1], inplace=True)
  return test_date

In [52]:
#test_date = generate_countinous('6/01/2018','7/01/2018')
test_date = generate_countinous('8/01/2018','9/01/2018')

In [ ]:
#test_date = generate_countinous('7/31/2018 23:46:00 ','8/01/2018 00:14:00')
test_date_06 = generate_countinous('6/30/2018 23:46:00 ','7/01/2018 00:14:00')
test_date_07 = generate_countinous('7/31/2018 23:46:00 ','8/01/2018 00:14:00')
#test_date

In [9]:
#extract the data slice that are wanted
#### really important stage for extraction ---- later
def data_extract(data, number):
  data_index = data[data['PULocationID'] == number].index.to_list()
  newData = data.iloc[data_index,:]
  return newData

In [0]:
cleaned_data = data_extract(newData, 237)

In [10]:
### try
def minutes_accumulate(cleaned_data):
  Counted_value = pd.DataFrame(cleaned_data.groupby('tpep_pickup_datetime', as_index=False).size())
  Counted_value.reset_index(drop= False, inplace = True)
  Counted_value.rename(columns = {0:'minute_demand'}, inplace = True)
  Counted_value['Sum_price'] = pd.Series(cleaned_data.groupby('tpep_pickup_datetime')['total_amount'].sum().values)
  Counted_value['Mean_price'] = pd.Series(cleaned_data.groupby('tpep_pickup_datetime')['total_amount'].mean().values)
  Counted_value['Sum_duration'] = pd.Series(cleaned_data.groupby('tpep_pickup_datetime')['duration'].sum().values)
  Counted_value['Mean_duration'] = pd.Series(cleaned_data.groupby('tpep_pickup_datetime')['duration'].mean().values)
  return Counted_value
#all(Counted_value.index == avg_price.index)

In [0]:
# merge the result and macth in minutes
cleaned_data = minutes_accumulate(cleaned_data)
matched_date = pd.merge(test_date, cleaned_data, left_on = 'pickup_time_g', right_on = 'tpep_pickup_datetime', how = 'left')

In [11]:
# if there is no order at this moment, we set this value to 999
#remove and fill in null values
def null_value_implaint(matched_date):
  del matched_date['tpep_pickup_datetime']
  matched_date['minute_demand'] = matched_date['minute_demand'].fillna(0)
  matched_date['Sum_price'] = matched_date['Sum_price'].fillna(0)
  matched_date['Mean_price'] = matched_date['Mean_price'].fillna(0)
  matched_date['Sum_duration'] = matched_date['Sum_duration'].fillna(0)
  matched_date['Mean_duration'] = matched_date['Mean_duration'].fillna(0)
  return matched_date

In [0]:
matched_date = null_value_implaint(matched_date)

In [12]:
def interval_extraction(matched_date, window_num):
    Demand = pd.Series(matched_date['minute_demand']).rolling(window = window_num).sum()
    Price = pd.Series(matched_date['Sum_price']).rolling(window = window_num).sum()
    Duration = pd.Series(matched_date['Sum_duration']).rolling(window = window_num).sum()
    #check the number of null values
    #tryit.isna().sum()
    Demand.dropna(inplace=True)
    Price.dropna(inplace=True)
    Duration.dropna(inplace=True)
    Demand.reset_index(drop=True,inplace = True)
    Price.reset_index(drop=True,inplace = True)
    Duration.reset_index(drop=True,inplace = True)
    cleaned_data = pd.concat([Demand,Price,Duration], axis=1)
    cleaned_data.rename(columns = {'minute_demand':'15_demand','Sum_price':'15_sum_price','Sum_duration':'15_sum_duration'},inplace = True)
    new = pd.concat([matched_date,cleaned_data], axis=1)
    new.dropna(inplace=True)
    return new

In [0]:
new = interval_extraction(matched_date, 15)

In [13]:
def separate_timestamp(matched_date):
    matched_date['weekday'] = matched_date['pickup_time_g'].dt.weekday
    matched_date['month'] = matched_date['pickup_time_g'].map(lambda x: x.month)
    matched_date['hour'] = matched_date['pickup_time_g'].map(lambda x: x.hour)
    matched_date['minute'] = matched_date['pickup_time_g'].map(lambda x: x.minute)
    matched_date['second'] = matched_date['pickup_time_g'].map(lambda x: x.second)
    matched_date['day'] = matched_date['pickup_time_g'].map(lambda x: x.day)
    return matched_date

In [0]:
matched_date = separate_timestamp(new)

In [14]:
def num_attach(dataset,number):
  locationID = [number] * len(matched_date)
  matched_date['locationID'] = pd.Series(locationID)
  return matched_date

In [16]:
cluster0_06 = [4, 11, 12, 14, 17, 21, 22, 24, 25, 26, 29, 33, 34, 35, 36, 37, 39, 40, 41, 42, 45, 49, 52, 54, 55, 61, 62, 63, 65, 66, 67, 71, 72, 74, 75, 76, 77, 80, 85, 88, 89, 91, 97, 106, 108, 111, 112, 116, 120, 123, 125, 127, 128, 133, 149, 150, 152, 153, 154, 155, 165, 166, 177, 178, 181, 188, 189, 190, 194, 195, 202, 209, 210, 217, 222, 224, 225, 227, 228, 232, 243, 244, 255, 256, 257, 261]
cluster1_06 = [13, 43, 50, 87, 90, 100, 113, 114, 137, 140, 141, 143, 144, 148, 151, 158, 211, 229, 231, 233, 238, 246, 249, 262, 263]
cluster2_06 = [48, 68, 79, 107, 142, 161, 162, 163, 164, 170, 186, 230, 234, 236, 237, 239]


In [ ]:
matched_or_not = pd.unique(newData['PULocationID'])
cluster0_06_in = list()
cluster0_06_out = list()
cluster1_06_in = list()
#void
cluster1_06_out = list()
cluster2_06_in = list()
#void 
cluster2_06_out = list()
for i in cluster0_06:
  if i in matched_or_not:
    cluster0_06_in.append(i)
  else:
    cluster0_06_out.append(i)
for i in cluster1_06:
    if i in matched_or_not:
       cluster1_06_in.append(i)
    else:
      cluster1_06_out.append(i)
for i in cluster2_06:
  if i in matched_or_not:
    cluster2_06_in.append(i)
  else:
    cluster2_06_out.append(i)

In [ ]:
taxi_zones = pd.read_csv('/Users/liuyang/Desktop/capstone/capstone_dataset/Geo_taxizone.csv')
taxi_zones = taxi_zones[['zone','borough','LocationID', 'longitude','latitude']]

In [55]:
def final_function(dataset, cluster_name,test_date):
  start = time.time()
  final_result = pd.DataFrame(columns = ['pickup_time_g', 'minute_demand', 'Sum_price', 'Mean_price',
       'Sum_duration', 'Mean_duration', '15_demand', '15_sum_price',
       '15_sum_duration', 'weekday', 'month', 'hour', 'minute', 'second',
       'day', 'locationID'])
  for number in cluster_name:   
    cleaned_data = data_extract(dataset, number)
    cleaned_data = minutes_accumulate(cleaned_data)
    matched_date = pd.merge(test_date, cleaned_data, left_on = 'pickup_time_g', right_on = 'tpep_pickup_datetime', how = 'left')
    matched_date = null_value_implaint(matched_date)
    new = interval_extraction(matched_date, 15)
    matched_date = separate_timestamp(new)
    key1 = num_attach(matched_date, number)
    final_result = pd.concat([final_result,key1],ignore_index = True)
  try_data1 = pd.Series(final_result['15_sum_duration']/final_result['15_demand'])
  final_result['15_avg_duration'] = try_data1.fillna(0)
  try_data2 = pd.Series(final_result['15_sum_price']/final_result['15_demand'])
  final_result['15_avg_price'] = try_data2.fillna(0)
  final_result.drop(columns = ['Sum_duration','Sum_price','15_sum_duration','15_sum_price'], inplace=True)
  final_result = pd.merge(final_result, taxi_zones, left_on = 'locationID', right_on = 'LocationID', how = 'left')
  final_result.drop(columns = ['locationID'], inplace = True)
  end = time.time()
  print(end-start)
  return final_result


/Users/liuyang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


18.72039818763733


In [ ]:
High_06 = final_function(newData,cluster0_06, test_date)
High_06_append = final_function(newData_07_append,cluster2_06_in, test_date_07)
Result = pd.concat([High_06,High_06_append], ignore_index=True)
Result.to_csv('/content/drive/capstone dataset/High_07.csv')
#High_06.to_csv('/content/drive/capstone dataset/High_07.csv')

In [ ]:
def format_columns(cluster, month):
    name = ['Low','Medium','High']
    test = pd.read_csv(f'/content/drive/capstone dataset/cluster{cluster}_0{month}.csv')
    test.drop(columns = ['Unnamed: 0'], inplace = True)
    cleaned = pd.merge(test, taxi_zones, left_on = 'locationID', right_on = 'LocationID', how = 'left')
    cleaned.drop(columns = ['locationID'], inplace = True)
    cleaned.to_csv(f'/content/drive/capstone dataset/{name[cluster]}_0{month}.csv')
    return cleaned

In [ ]:
a = [0,1,2]
b = [6,7,8]
for i in a:
    for j in b:
        format_columns(i, j)

In [85]:
def concat_summer_data(number):
    b = ['LowDemand','MediumDemand','HighDemand']
    Low_06 = pd.read_csv(f'/Users/liuyang/Desktop/formated_data/cluster{number}_06.csv')
    Low_07 = pd.read_csv(f'/Users/liuyang/Desktop/formated_data/cluster{number}_07.csv')
    Low_08 = pd.read_csv(f'/Users/liuyang/Desktop/formated_data/cluster{number}_08.csv')
    result = pd.concat(['Low_06','Low_07','Low_08'], ignore_index = True)
    result.drop(columns = ['Unnamed: 0'], inplace = True)
    result.to_csv(f'/Users/liuyang/Desktop/{b[number]}_3months.csv')
    return result